In [1]:
import netCDF4
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, IntSlider, Button
import sys
import subprocess
import os
sys.path.append('../src')
from create_animations_delft3dfm import extract_and_save_transect_animation, get_timestamps, get_closest_index_by_coord, get_closest_flow_element_index, interpolate_points_on_line, extract_and_save_xy_animation, plot_transect, get_transect_direction

## Parameters plot

In [2]:
name_simu = r'test_dummy\buoyancy_flux' # grosserbrombachsee run_004_ope_grid test_dummy\seiche
working_dir = rf'C:\Users\leroquan\Documents\00-Work_space\01-Lake-Lugano_Delft3D-FM\{name_simu}'
#working_dir = r'C:\Users\leroquan\Documents\GIT\alplakes-simulations\runs\eawag_delft3dflow60210142612_delft3dfm_lugano_20230201_20240201'

In [ ]:
delft3Dfm_exe_path = r"C:\Program Files\Deltares\Delft3D FM Suite 2023.02 HMWQ\plugins\DeltaShell.Dimr\kernels\x64\dimr\scripts\run_dimr.bat"
dimr_file_path = os.path.join(working_dir, 'dimr_config.xml')

process = subprocess.Popen([delft3Dfm_exe_path, dimr_file_path],
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE,
                            universal_newlines=True,
                            cwd=working_dir)
process.communicate()

In [3]:
output_file = rf"{working_dir}\dflowfm\output\FlowFM_map.nc"
output_data = netCDF4.Dataset(output_file)
var_to_plot = 'ucx'
index_time_to_plot = 5
i_depth = 179 # TODO : find depth corresponding to i_depth
x_to_plot = 500 # 729429.4
y_to_plot = 500 # 98326.6

In [149]:
# tem1(time, nFlowElem, laydim) : water temperature
# ucx(time, nFlowElem, laydim) : velocity on flow element center, x-component
# ucy(time, nFlowElem, laydim) : velocity on flow element center, x-component
# ucz(time, nFlowElem, laydim) : upward velocity on flow element center
# taus('time', 'nFlowElem') : taucurrent in flow element"
# czs('time', 'nFlowElem') Chezy roughness",

## Get grid and time properties

In [4]:
x_flowElems = output_data['FlowElem_xzw'][:].data #FlowElem_xzw(nFlowElem) : x-coordinate of flow element center of mass
y_flowElems = output_data['FlowElem_yzw'][:].data #FlowElem_yzw(nFlowElem) : y-coordinate of flow element center of mass
z_flowElems = output_data['FlowElem_zcc'][0].data[0,:] #FlowElem_zcc(nFlowElem) : bed level of flow element

IndexError: FlowElem_xzw not found in /

In [ ]:
timestamps = get_timestamps(output_data)

In [5]:
i_nElem = get_closest_index_by_coord(x_flowElems, y_flowElems, x_to_plot, y_to_plot)
pattern = index_time_to_plot, slice(None), i_depth 
pattern_timeserie = slice(None), i_nElem, i_depth
if  var_to_plot in ['czs', 'taus']:
    pattern = index_time_to_plot, slice(None)
    pattern_timeserie = slice(None), i_nElem

NameError: name 'x_flowElems' is not defined

## Get results

In [ ]:
data_to_plot = output_data[var_to_plot][:].data
data_to_plot[data_to_plot == -999] = np.nan

## Plot results

def plot_transect(i_time):
    fig = plt.figure(figsize=(15, 7))
    data_plot_transect = data_to_plot[i_time, transect_flow_elems, slice(None)].transpose()
    l_coords = [np.arange(len(transect_flow_elems)) * distance_between_points]
    z_coords = z_flowElems
    L, dummy = np.meshgrid(l_coords, z_coords)
    # Bricolage to fix z values for Nan cells
    Z = output_data['FlowElem_zcc'][0].data[transect_flow_elems,:].transpose()
    indices = np.where(Z == -999)
    Z[Z==-999] = np.nan
    Z[indices] = (len(z_flowElems)-indices[0] ) * np.nanmin(Z) / len(z_flowElems)
    v_min = np.nanmin(data_to_plot[slice(None), transect_flow_elems, slice(None)])
    v_max = np.nanmax(data_to_plot[slice(None), transect_flow_elems, slice(None)])
    
    plt.pcolormesh(L, Z, data_plot_transect, cmap='coolwarm', vmin=v_min, vmax=v_max)
    plt.title(var_to_plot + " at time: " + str(timestamps[i_time]))
    plt.colorbar()

In [ ]:
def project_speed(u, v, xy1, xy2):
    if np.isnan(u) or np.isnan(v):
        return np.nan
    # Calculate direction vector
    dx = xy2[0] - xy1[0]
    dy = xy2[1] - xy1[1]
    # Normalize direction vector
    mag = np.sqrt(dx**2 + dy**2)
    if mag == 0:  # Prevent division by zero
        return np.nan
    dir_x = dx / mag
    dir_y = dy / mag
    # Calculate dot product
    projection = u * dir_x + v * dir_y
    return projection

In [ ]:
xy_transect_start = [701000,87000]
xy_transect_end = [728000,87000]
interpolated_points, step_length = interpolate_points_on_line(xy_transect_start, xy_transect_end)
transect_flow_elems = []
for point in interpolated_points:
    transect_flow_elems.append(int(get_closest_flow_element_index(x_flowElems, y_flowElems, point[0], point[1])))

dir_x, dir_y = get_transect_direction(xy_transect_start, xy_transect_end)

def increment_time(button):
    time_slider_transect.value += 1
    
button_increment = Button(description='Increment Time')
button_increment.on_click(increment_time)

def update_plot_transect(i_time):
    fig, ax = plt.subplots(figsize=(15, 7)) 
    ax.clear()
    img = plot_transect(ax, i_time, output_data, data_to_plot, transect_flow_elems,
                                  step_length, z_flowElems, timestamps, var_to_plot)
    cbar = fig.colorbar(ax.collections[0], ax=ax)
    plt.show()

time_slider_transect = IntSlider(min=0, max=len(timestamps)-1, step=1, value=0, continuous_update=True)
interact(update_plot_transect, i_time=time_slider_transect)
display(button_increment)

In [ ]:
def plot_xy(i_time, data_pattern, data, x_values, y_values, title, vmin, vmax):
    plt.figure(figsize=(15, 7))
    new_pattern = (i_time,) + data_pattern[1:]
    scatter = plt.scatter(x_values, y_values, s=50, c=data[new_pattern], cmap='coolwarm', vmin=vmin, vmax=vmax)
    plt.colorbar(scatter, label='Value')
    plt.title(title)
    plt.show()

In [ ]:
def plot_xy_with_vectors(i_time, data_pattern, var_data, x_values, y_values, title, vmin, vmax, index_depth, data):
    plt.figure(figsize=(15, 7))
    new_pattern = (i_time,) + data_pattern[1:]
    scatter = plt.scatter(x_values, y_values, s=50, c=var_data[new_pattern], cmap='coolwarm', vmin=vmin, vmax=vmax)
    plt.quiver(x_values, y_values, data['ucx'][:].data[i_time, slice(None), index_depth],data['ucy'][:].data[i_time, slice(None), index_depth])
    plt.colorbar(scatter, label='Value')
    plt.title(title)
    plt.show()

In [ ]:
time_slider = IntSlider(min=0, max=len(timestamps)-1, step=1, value=0, continuous_update=True)
title_plot_xy = f'{var_to_plot} at time {str(timestamps[index_time_to_plot])}, depth = {round(z_flowElems[i_depth],2)}'
v_min = np.nanmin(data_to_plot[(slice(None),) + pattern[1:]]) 
v_max = np.nanmax(data_to_plot[(slice(None),) + pattern[1:]])

def plot_xy_slider(i_time_slider):
    plot_xy(i_time_slider, pattern, data_to_plot, x_flowElems, y_flowElems, title_plot_xy, v_min, v_max)

def increment_time_xy(button):
    time_slider.value += 1
    
button_increment_xy = Button(description='Increment Time')
button_increment_xy.on_click(increment_time_xy)

interact(plot_xy_slider, i_time_slider=time_slider)
display(button_increment_xy)

## Save Animation

output_animation_path = r'C:\Users\leroquan\Documents\00-Work_space\01-Lake-Lugano_Delft3D-FM\test_dummy\seiche\animation.gif'

extract_and_save_transect_animation(output_file, output_animation_path, var_to_plot, xy_transect_start, xy_transect_end, distance_between_points)

output_animation_path = r'C:\Users\leroquan\Documents\00-Work_space\01-Lake-Lugano_Delft3D-FM\test_dummy\seiche\animation.gif'

extract_and_save_xy_animation(output_file, output_animation_path, var_to_plot, i_depth)

## Display speed vectors

In [ ]:
def plot_speed_vectors(data, i_time, index_depth, x_values, y_values, title):
    plt.figure(figsize=(15, 7))
    
    plt.title(title)
    plt.show()

title_plot_speed_vectors =  f'Horizontale speed at time {str(timestamps[index_time_to_plot])}, depth = {round(z_flowElems[i_depth],2)}'
plot_speed_vectors(output_data, 10, i_depth, x_flowElems, y_flowElems, title_plot_speed_vectors)

## Analyse inputs

input_grid_file = r"C:\Users\leroquan\Documents\00-Work_space\01-Lake-Lugano_Delft3D-FM\test_dummy\dimr\dflowfm\FlowFM_net.nc"
grid_data = netCDF4.Dataset(input_grid_file)

grid_data

grid_data2['mesh2d_node_z'][:].data

bath_file_path = r"C:\Users\leroquan\Documents\00-Work_space\01-Lake-Lugano_Delft3D-FM\run_005\dflowfm\bathymetry_fill.xyz"
bath = open(bath_file_path, 'r')

corr_bath_path = r"C:\Users\leroquan\Documents\00-Work_space\01-Lake-Lugano_Delft3D-FM\run_005\dflowfm\corr_bath.xyz"
with open(corr_bath_path, 'w') as f:
    for line in bath:        
        coords = line.split('   ')
        depth = coords[3]
        print(coords)
        corr_depth = - float(depth)
        print(corr_depth)
        line_to_write = '   ' + coords[1] + '   ' + coords[2] + '   ' + str(corr_depth) + '\n'
        print(line_to_write)
        f.write(line_to_write)
    

## Analyse results variables

var_infos = {}
for var_name in output_data.variables.keys():
    var = output_data.variables[var_name]
    long_name = ''
    if 'long_name' in var.ncattrs():
        long_name = var.getncattr('long_name')
    var_infos[var_name] = f"{var.dimensions} {long_name}"